In [3]:
import pandas as pd
import spacy 

nlp = spacy.load("en_core_sci_md")
txt_col : str = "transcription"

data_path : str = "../data/mtsamples.csv"



def get_vectors(doc) -> float:
    """Get Vectors from text using Spacy. 

    Vectors are obtained after stopwords removal.
    A simple vector avg. is calculated for a given sentence.
    
    Args:
        doc (spacy.tokens.doc.Doc): Document processed by spacy's pipeline
    Returns:
        np.ndarray: mean of the vector representation of the sentence.
    """
    cleaned_txt = " ".join([token.lemma_ for token in doc if not token.is_stop])


    return nlp(cleaned_txt).vector


In [9]:
df = pd.read_csv(data_path)
# drop nas from the text column
df = df[df[txt_col].notna()].reset_index(drop=True)
# Get only Surgery topics 
df = df[df["medical_specialty"] == " Surgery"]

In [10]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
172,174,Austin & Youngswick bunionectomy with Biopro ...,Surgery,Youngswick Bunionectomy,"PREOPERATIVE DIAGNOSES:,1. Hallux rigidus, le...","surgery, hallux rigidus, metatarsal, youngswic..."
175,177,"This patient has undergone cataract surgery, ...",Surgery,YAG Laser Capsulotomy - 1,"PREOPERATIVE DIAGNOSIS: , Secondary capsular m...","surgery, abraham capsulotomy, yag, yag laser c..."
177,179,Youngswick osteotomy with internal screw fixa...,Surgery,Youngswick Osteotomy,"TITLE OF OPERATION: , Youngswick osteotomy wit...","surgery, hallux limitus deformity, metatarsoph..."
179,181,Wound debridement with removal of Surgisis xe...,Surgery,Wound Debridement,"PREOPERATIVE DIAGNOSES,1. Open wound from rig...","surgery, open wound, prosthetic vascular graft..."
180,182,Visually significant posterior capsule opacit...,Surgery,YAG Laser Capsulotomy,"PREOPERATIVE DIAGNOSIS:, Visually significant...","surgery, capsule opacity, yag, ophthalmic, yag..."


In [12]:
df["vectors"] = df[txt_col].apply(lambda sent : get_vectors(nlp(sent)))